In [14]:
import markdown
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from sentence_transformers import SentenceTransformer


/opt/anaconda3/envs/czb_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [22]:
nlp = spacy.load("en_core_web_md")